<div style="text-align: center;">
    <h1 style="color: blue;">Anexo 9</h1>
    <h3>Proyecto 9: Generador de texto</h3>
    <hr/>
    <p style="text-align: right;">Mg. Luis Felipe Bustamante Narváez</p>
</div>

En este ejercicio realizaremos un generador de texto, basado en el archivo pdf trabajado en el clasificador <span style='color:red;'>mario-benedetti.pdf</span>. Este archivo, nos permitirá entrenar un modelo de <b style='color:red;'>Markov</b> de <b style='color:red;'>Segundo Orden</b>.

Tengamos en cuenta los siguientes detalles de la manera de escribir del autor:

<div align="center">
    <table>
        <tr>
            <td align="center">
                <img src="imagenes/mario-benedetti.jpg" alt="Mario Benedetti" width="211"><br>
                <strong>Mario Benedetti</strong>
            </td>            
        </tr>
        <tr>
        <td style='color:gray; text-align:center;'>
            Mario Benedetti (1920-2009), fue un escritor, poeta y periodista uruguayo, 
            considerado una de las figuras más importantes de la literatura latinoamericana.
            Nació el 14 de septiembre de 1920 en Paso de los Toros, Uruguay. Su obra abarcó
            poesía, narrativa, ensayo y teatro, con un estilo sencillo y cercano que
            exploraba el amor, la memoria, el exilio y la lucha social.
            Durante la dictadura en Uruguay (1973-1985), se exilió en varios países como
            Argentina, Cuba y España, lo que marcó profundamente su escritura. Entre sus
            libros más conocidos se encuentran La tregua (1960), Gracias por el fuego (1965)
            y El amor, las mujeres y la vida (1995).
            Benedetti falleció el 17 de mayo de 2009 en Montevideo, dejando un legado
            literario que sigue conmoviendo a lectores de todas las generaciones.
        </td>            
        </tr>
    </table>
</div>

**📖 1. Temas y Filosofía**<br><br>
Escribe de manera directa y accesible. Sus temas son el amor, la vida cotidiana, la lucha social, el exilio y la esperanza. Su tono es cálido, humano y cercano al lector.

**🖋 2. Lenguaje y Estilo**<br><br>
Usa un lenguaje sencillo, directo y coloquial. Sus poemas parecen conversaciones o pensamientos escritos sin mucha ornamentación.

**🔄 3. Estructura y Ritmo**<br><br>
Prefiere el verso libre y la naturalidad del habla cotidiana, sin preocuparse demasiado por la métrica.



## Librerías

In [65]:
import numpy as np
import string
import PyPDF2
import itertools

## Convertimos el pdf en .txt

En esta oportunidad, como ya conocemos el manejo de pdf's, vamos a usar un conversor a .txt, para agilizar la manipulación de los datos.

In [41]:
def pdf_to_text(pdf_path, txt_path):
    with open(pdf_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

    print(f'Texto extraído y guardado en "{txt_path}"')

In [42]:
# Ruta pdf
pdf = 'textos/mario_benedetti.pdf'
txt = 'textos/mario_benedetti.txt'
pdf_to_text(pdf, txt)

Texto extraído y guardado en "textos/mario_benedetti.txt"


## Diccionarios

### Diccionario inicial

<p>Supongamos que tenemos la siguiente frase:</p>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
  <div style="padding: 10px; border: 2px solid #3498db; background: #ecf0f1; border-radius: 5px; text-align: center;">la</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #3498db;">→</div>
  <div style="padding: 10px; border: 2px solid #e74c3c; background: #f9ebea; border-radius: 5px; text-align: center;">casa</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #e74c3c;">→</div>
  <div style="padding: 10px; border: 2px solid #2ecc71; background: #eafaf1; border-radius: 5px; text-align: center;">del</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #2ecc71;">→</div>
  <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">perro</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #f1c40f;">→</div>
  <div style="padding: 10px; border: 2px solid #9b59b6; background: #f5eef8; border-radius: 5px; text-align: center;">?</div>
</div>

<hr>

<p>Necesitamos obtener la primera palabra de cada frase, y las veces que dicha palabra aparece dentro del texto como primera palabra.</p>
<p>Por ejemplo, en el diagrama anterior, la palabra <b style="padding: 10px; border: 2px solid #3498db; background: #ecf0f1; border-radius: 5px; text-align: center;">la</b> aparece como primera palabra de la frase <b style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">la casa del perro </b> si suponemos que es la única frase de nuestro texto, entonces dicha palabra se repite <b style="padding: 10px; border: 2px solid #2ecc71; background: #eafaf1; border-radius: 5px; text-align: center;">1</b> vez.</p>

<p>Luego, el diccionario inicial quedaría así:</p>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 400px">
  <div style="padding: 10px; border: 2px solid #e74c3c; background: #f9ebea; border-radius: 5px; text-align: center;">pa_inicial = { 'la': 1 }</div>
</div>
<hr>

In [43]:
pa_inicial = {}

### Diccionario de Markov Primero Orden

<p>Supongamos que tenemos las siguientes frases:</p>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
  <div style="padding: 10px; border: 2px solid #3498db; background: #ecf0f1; border-radius: 5px; text-align: center;">la</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #3498db;">→</div>
  <div style="padding: 10px; border: 2px solid #e74c3c; background: #f9ebea; border-radius: 5px; text-align: center;">casa</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #e74c3c;">→</div>
  <div style="padding: 10px; border: 2px solid #2ecc71; background: #eafaf1; border-radius: 5px; text-align: center;">del</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #2ecc71;">→</div>
  <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">perro</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #f1c40f;">→</div>
  <div style="padding: 10px; border: 2px solid #9b59b6; background: #f5eef8; border-radius: 5px; text-align: center;">?</div>
</div>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
  <div style="padding: 10px; border: 2px solid #3498db; background: #ecf0f1; border-radius: 5px; text-align: center;">el</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #3498db;">→</div>
  <div style="padding: 10px; border: 2px solid #e74c3c; background: #f9ebea; border-radius: 5px; text-align: center;">vino</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #e74c3c;">→</div>
  <div style="padding: 10px; border: 2px solid #2ecc71; background: #eafaf1; border-radius: 5px; text-align: center;">está</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #2ecc71;">→</div>
  <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">frío</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #f1c40f;">→</div>
  <div style="padding: 10px; border: 2px solid #9b59b6; background: #f5eef8; border-radius: 5px; text-align: center;">?</div>
</div>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
  <div style="padding: 10px; border: 2px solid #3498db; background: #ecf0f1; border-radius: 5px; text-align: center;">la</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #3498db;">→</div>
  <div style="padding: 10px; border: 2px solid #e74c3c; background: #f9ebea; border-radius: 5px; text-align: center;">viuda</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #e74c3c;">→</div>
  <div style="padding: 10px; border: 2px solid #2ecc71; background: #eafaf1; border-radius: 5px; text-align: center;">del</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #2ecc71;">→</div>
  <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">general</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #f1c40f;">→</div>
  <div style="padding: 10px; border: 2px solid #9b59b6; background: #f5eef8; border-radius: 5px; text-align: center;">?</div>
</div>
<hr>


<p>Necesitamos obtener la primera palabra de cada frase, y las veces que dicha palabra aparece dentro del texto como primera palabra, como vimos anteriormente, además necesitamos saber cuales son las secuencias de primer orden, para componer nuestro diccionario.</p>

<p>Luego, el diccionario inicial y el diccionario de primer orden quedarían así:</p>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 400px">
  <div style="padding: 10px; border: 2px solid #e74c3c; background: #f9ebea; border-radius: 5px; text-align: center;">pa_inicial = { 'la': 2, 'el':1 }</div>
</div>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
  <div style="padding: 10px; border: 2px solid #9b59b6; background: #f5eef8; border-radius: 5px; text-align: right;">primer_orden = { 'la': ['casa', 'viuda'],<br>
          'casa': ['del'],<br>
          'del': ['perro', 'general'],<br>
          'perro': ['END'],<br>
          'el': ['vino'],<br>
          'vino': ['frío'],<br>
          'frío': ['END'],<br>
          'viuda': ['del'],<br>
          'general': ['END']<br>
          }</div>
</div>
<hr>

In [44]:
primer_orden = {}

### Diccionario de Markov Segundo Orden

<p>Supongamos para el ejemplo anterior:</p>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
  <div style="padding: 10px; border: 2px solid #3498db; background: #ecf0f1; border-radius: 5px; text-align: center;">la</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #3498db;">→</div>
  <div style="padding: 10px; border: 2px solid #e74c3c; background: #f9ebea; border-radius: 5px; text-align: center;">casa</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #e74c3c;">→</div>
  <div style="padding: 10px; border: 2px solid #2ecc71; background: #eafaf1; border-radius: 5px; text-align: center;">del</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #2ecc71;">→</div>
  <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">perro</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #f1c40f;">→</div>
  <div style="padding: 10px; border: 2px solid #9b59b6; background: #f5eef8; border-radius: 5px; text-align: center;">?</div>
</div>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
  <div style="padding: 10px; border: 2px solid #3498db; background: #ecf0f1; border-radius: 5px; text-align: center;">el</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #3498db;">→</div>
  <div style="padding: 10px; border: 2px solid #e74c3c; background: #f9ebea; border-radius: 5px; text-align: center;">vino</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #e74c3c;">→</div>
  <div style="padding: 10px; border: 2px solid #2ecc71; background: #eafaf1; border-radius: 5px; text-align: center;">está</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #2ecc71;">→</div>
  <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">frío</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #f1c40f;">→</div>
  <div style="padding: 10px; border: 2px solid #9b59b6; background: #f5eef8; border-radius: 5px; text-align: center;">?</div>
</div>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
  <div style="padding: 10px; border: 2px solid #3498db; background: #ecf0f1; border-radius: 5px; text-align: center;">la</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #3498db;">→</div>
  <div style="padding: 10px; border: 2px solid #e74c3c; background: #f9ebea; border-radius: 5px; text-align: center;">viuda</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #e74c3c;">→</div>
  <div style="padding: 10px; border: 2px solid #2ecc71; background: #eafaf1; border-radius: 5px; text-align: center;">del</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #2ecc71;">→</div>
  <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">general</div>
  <div style="margin: 10px 10px; font-size: 20px; color: #f1c40f;">→</div>
  <div style="padding: 10px; border: 2px solid #9b59b6; background: #f5eef8; border-radius: 5px; text-align: center;">?</div>
</div>
<hr>


<p>Necesitamos el conjunto de palabras que anteceden a una palabra cualquiera, para ello, usamos un diccionario donde la clave sean las dos palabras inmediatamente anteriores en el tiempo.</p>

<p>Luego, el diccionario de segundo orden quedaría así:</p>

<div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
  <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: right;">segundo_orden = { ('la', 'casa'): ['del'],<br>
          ('casa', 'del'): ['perro'],<br>
          ('el', 'vino'): ['está'],<br>
          ('vino', 'está'): ['frío'],<br>
          ('la', 'viuda'): ['del'],<br>
          ('viuda', 'del'): ['general'],<br>          
          }</div>
</div>
<hr>
<p>Notemos, que las claves de cada diccionario, son conjuntos de palabras.</p>


In [45]:
segundo_orden = {}

In [46]:
# Función para remover puntuación y poner en minúsculas
def remove_punct_lower(txt):
    txt = txt.translate(str.maketrans('','', string.punctuation))
    txt = txt.lower()
    return txt

In [47]:
# Función para añadir valores al diccionario
def add_dict(dicc, key, value):
    if key not in dicc:
        dicc[key]= []
    dicc[key].append(value)

## Cargamos el archivo

In [60]:
# Reiniciar diccionarios
pa_inicial = {}
primer_orden = {}
segundo_orden = {}

In [61]:
# Si necesitamos reiniciar este código para efectos prácticos, basta con reiniciar los dict
with open('textos/mario_benedetti.txt', 'r', encoding='utf-8') as archivo:
    for linea in archivo:
        #print(linea)  #comentar
        #llamamos a la función 
        tokens = remove_punct_lower(linea).split()
        #print(tokens)  #comentar
        T = len(tokens)
        #print(f'Tamaño de la línea: {T}')  #Comentar
        #Recorremos los elementos de la fila
        for i in range(T):
            token = tokens[i]
            if i == 0:
                pa_inicial[token] = pa_inicial.get(token, 0.) + 1
                #print(f'Palabra inicial: {token}') #comentar
            else:
                t_1 = tokens[i-1]
                #últimas 2 palabras de cada frase
                if i == T-1:
                    add_dict(segundo_orden, (t_1, token), 'END')
                #palabras con una sola palabra previa
                if i == 1:
                    add_dict(primer_orden, t_1, token)
                else:
                    t_2 = tokens[i-2]
                    add_dict(segundo_orden, (t_2, t_1), token)
                

In [86]:
dict(itertools.islice(pa_inicial.items(), 5))

{'poemas': 3.0, 'mario': 1.0, 'la': 26.0, 'una': 16.0, 'genera': 1.0}

In [85]:
dict(itertools.islice(segundo_orden.items(), 5))

{('poemas', 'varios'): ['END'],
 ('mario', 'benedetti'): ['END'],
 ('buena', 'tiniebla'): ['END'],
 ('la', 'buena'): ['tiniebla', 'fe', 'suerte'],
 ('una', 'mujer'): ['desnuda', 'desnuda', 'desnuda', 'querida', 'dice']}

In [84]:
dict(itertools.islice(primer_orden.items(), 3))

{'poemas': ['varios', 'a', 'al'],
 'mario': ['benedetti'],
 'la': ['buena',
  'madriguera',
  'lluvia',
  'vida',
  'patria',
  'política',
  'claridad',
  'primavera',
  'misma',
  'luz',
  'válida',
  'madre',
  'incógnita',
  'pareja',
  'pareja',
  'despareja',
  'gente',
  'cosa',
  'querida',
  'culpa',
  'culpa',
  'ebriedad',
  'política',
  'muerte',
  'generosidad',
  'soledad']}

### Explicación

Analicemos los resultados de los cinco primeros elementos de cada uno de los diccionarios creados a partir del texto.

1. pa_inicial:
 
     <p>
        <span style="color: black">{</span>
        <span style="color: purple">'poemas'</span>: <span style="color: green">3.0</span>,
        <span style="color: purple">'mario'</span>: <span style="color: green">1.0</span>,
        <span style="color: purple">'la'</span>: <span style="color: green">26.0</span>,
        <span style="color: purple">'una'</span>: <span style="color: green">16.0</span>,
        <span style="color: purple">'genera'</span>: <span style="color: green">1.0</span>
        <span style="color: black">}</span>
    </p>

   Indica, por ejemplo que, la palabra <b style="color: purple">la</b> aparece como palabra inicial <b style="color: green">16.0</b>.

2. primer_orden:

   <pre>
        <span style="color: black">{</span>
        <span style="color: purple">'poemas'</span>: <span style="color: black">[</span>
            <span style="color: green">'varios'</span>, 
            <span style="color: green">'a'</span>, 
            <span style="color: green">'al'</span>
        <span style="color: black">]</span>,
        <span style="color: purple">'mario'</span>: <span style="color: black">[</span>
            <span style="color: green">'benedetti'</span>
        <span style="color: black">]</span>,
        <span style="color: purple">'la'</span>: <span style="color: black">[</span>
            <span style="color: green">'buena'</span>,
            <span style="color: green">'madriguera'</span>,
            <span style="color: green">'lluvia'</span>,
            <span style="color: green">'vida'</span>,
            .
            .
            .
            <span style="color: green">'política'</span>,
            <span style="color: green">'muerte'</span>,
            <span style="color: green">'generosidad'</span>,
            <span style="color: green">'soledad'</span>
        <span style="color: black">]</span>
        <span style="color: black">}</span>
    </pre>

   Indica, por ejemplo que, después de la palabra <span style="color: purple">'la'</span> aparecen las siguientes palabras: <span style="color: green">'buena', 'madriguera', 'lluvia', etc.</span>.

3. segundo_orden:

   <pre>
        <span style="color: black">{</span>
        <span style="color: fuchsia">(</span><span style="color: purple">'poemas'</span>, <span style="color: purple">'varios'</span><span style="color: fuchsia">)</span>: <span style="color: black">[</span><span style="color: green">'END'</span><span style="color: black">]</span>,
        <span style="color: fuchsia">(</span><span style="color: purple">'mario'</span>, <span style="color: purple">'benedetti'</span><span style="color: fuchsia">)</span>: <span style="color: black">[</span><span style="color: green">'END'</span><span style="color: black">]</span>,
        <span style="color: fuchsia">(</span><span style="color: purple">'buena'</span>, <span style="color: purple">'tiniebla'</span><span style="color: fuchsia">)</span>: <span style="color: black">[</span><span style="color: green">'END'</span><span style="color: black">]</span>,
        <span style="color: fuchsia">(</span><span style="color: purple">'la'</span>, <span style="color: purple">'buena'</span><span style="color: fuchsia">)</span>: <span style="color: black">[</span>
            <span style="color: green">'tiniebla'</span>, 
            <span style="color: green">'fe'</span>, 
            <span style="color: green">'suerte'</span>
        <span style="color: black">]</span>,
        <span style="color: fuchsia">(</span><span style="color: purple">'una'</span>, <span style="color: purple">'mujer'</span><span style="color: fuchsia">)</span>: <span style="color: black">[</span>
            <span style="color: green">'desnuda'</span>, 
            <span style="color: green">'desnuda'</span>, 
            <span style="color: green">'desnuda'</span>, 
            <span style="color: green">'querida'</span>, 
            <span style="color: green">'dice'</span>
        <span style="color: black">]</span>
        <span style="color: black">}</span>
    </pre>

   Indica, por ejemplo que, después de las palabras <span style="color: fuchsia">(</span><span style="color: purple">'la', 'buena'</span><span style="color: fuchsia">)</span> aparecen las siguientes palabras: <span style="color: green">'tiniebla', 'fe' y 'suerte'</span>.

## Probabilidades

### Normalización

#### Palabras iniciales

In [88]:
# Calculamos el total de apariciones de todas las palabras iniciales
inicial_total = sum(pa_inicial.values())
inicial_total

1474.0

In [75]:
# Creamos el diccionario inicial de probabilidad
pa_inicial_prob = pa_inicial.copy()
for key, value in pa_inicial.items():
    pa_inicial_prob[key] = value / inicial_total

In [83]:
dict(itertools.islice(pa_inicial_prob.items(), 3))

{'poemas': 0.0020352781546811396,
 'mario': 0.0006784260515603799,
 'la': 0.017639077340569877}

#### Función de conversión a probabilidad

In [77]:
# Función para convertir las listas de primer orden en diccionarios
# 'poemas': ['varios', a', 'al'],
def list_to_pdicc(listas):
    # Creamos un diccionario vacío
    d = {}
    # Obtenemos la longitud de la lista de elementos
    n = len(listas)

    #Ciclo para contar la ocurrencia de cada elemento en la lista
    for lista in listas:
        d[lista] = d.get(lista, 0.) + 1

    # Ciclo para cnvertir los conteos en probabilidades relativas
    for key, value in d.items():
        d[key] = value / n

    # Retornamos el diccionario de probabilidades
    return d

#### Primer Orden

In [79]:
# Llamamos la función de probabilidad de primer orden
primer_orden_prob = primer_orden.copy()
for t_1, t in primer_orden.items():
    primer_orden_prob[t_1] = list_to_pdicc(t)

In [87]:
dict(itertools.islice(primer_orden_prob.items(), 2))

{'poemas': {'varios': 0.3333333333333333,
  'a': 0.3333333333333333,
  'al': 0.3333333333333333},
 'mario': {'benedetti': 1.0}}

#### Segundo Orden

In [89]:
# Llamamos la función de probabilidad de segundo orden
segundo_orden_prob = segundo_orden.copy()
for s, t in segundo_orden.items():
    segundo_orden_prob[s] = list_to_pdicc(t)    

In [92]:
dict(itertools.islice(segundo_orden_prob.items(), 5))

{('poemas', 'varios'): {'END': 1.0},
 ('mario', 'benedetti'): {'END': 1.0},
 ('buena', 'tiniebla'): {'END': 1.0},
 ('la', 'buena'): {'tiniebla': 0.3333333333333333,
  'fe': 0.3333333333333333,
  'suerte': 0.3333333333333333},
 ('una', 'mujer'): {'desnuda': 0.6, 'querida': 0.2, 'dice': 0.2}}

### Función de prueba

Con esta función, realizaremos las pruebas necesarias para evaluar el comportamiento de las probabilidades de aparición de una palabra. 

La función recibe dos parámetros, <b style='color:red;'>diccionario</b> e <b style='color:blue;'>imprimir</b>; el primer parámetro recibe el diccionario de trabajo a evaluar, y el segundo recibe un booleano, para saber si mostramos o no los pasos de la ejecución.

In [173]:
def palabra_ejemplo(d, imp):
    # Generamos un número aleatorio en el rango [0, 1]
    p0 = np.random.random()
    if imp:
        print(f"p0:\t\t{p0}")

    # Inicializamos una variable para realizar la suma acumulativa de probabilidades
    cumulative = 0
    if imp:
        print(f"prob. acum.:\t{cumulative}")

    # Ciclo que recorre cada clave (k) y su probabilidad (p) en el diccionario (d)
    for k, p in d.items():
        cumulative += p
        if imp:
            print(f"Prob:\t\t{p}\titem:\t'{k}'")
            print(f"prob. acum.:\t{cumulative}")

        # Comprobamos si el número aleatorio es menor que la acumulación de probabilidades
        if p0 < cumulative:
            # Si la condición se cumple, devuelve la clave (k) seleccionada
            respuesta = f"La palabra siguiente debería ser '{k}'"
            return respuesta if imp else k

In [99]:
# Verificamos que nos traería una palabra en particular
print(primer_orden_prob['poemas'])

{'varios': 0.3333333333333333, 'a': 0.3333333333333333, 'al': 0.3333333333333333}


In [174]:
# Llamamos la función
palabra_ejemplo(primer_orden_prob['poemas'], True)

p0:		0.36180865610514357
prob. acum.:	0
Prob:		0.3333333333333333	item:	'varios'
prob. acum.:	0.3333333333333333
Prob:		0.3333333333333333	item:	'a'
prob. acum.:	0.6666666666666666


"La palabra siguiente debería ser 'a'"

## Generador

### Función del generador de texto

In [175]:
def generador(tamaño):
    for i in range(tamaño):
        oracion = []
        #Palabra inicial
        pal_0 = palabra_ejemplo(pa_inicial_prob, False)
        oracion.append(pal_0)
        #segunda palabra
        pal_1 = palabra_ejemplo(primer_orden_prob[pal_0], False)
        oracion.append(pal_1)

        # Segundo orden hasta el final
        while True:
            pal_2 = palabra_ejemplo(segundo_orden_prob[(pal_0, pal_1)], False)
            if pal_2 == 'END':
                break
            oracion.append(pal_2)
            pal_0 = pal_1
            pal_1 = pal_2
        texto = ' '.join(oracion)
        print(texto)

### Explicación
<ul>
<li>Esta función recibe un parámetro tamaño, que representa la cantidad de oraciones que se generarán:
    <div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
        <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">
            <span style='color:green;'>def</span>
            <span style='color:blue;'>generador</span>
            (tamaño):
        </div>
    </div>
</li>
<hr>
<li>Se inicia un bucle que se ejecutará <b>tamaño</b> veces, es decir, generará <b>tamaño</b> oraciones.
    <div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
        <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">
            <span style='color:green;'>for</span>
            i
            <span style='color:green;'>in range</span>
            (tamaño):
        </div>
    </div>
</li>
<hr>
<li>Se crea una lista vacía llamada oracion, donde se almacenarán las palabras generadas.
    <div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 400px">
        <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">
            oracion
            <span style='color:fuchsia;'>=</span>            
            <span style='color:brown;'>[]</span>
        </div>
    </div>
</li>
<hr>
<li>Se elige la primera palabra de la oración con la función <b>palabra_ejemplo(pa_inicial_prob, False)</b>. <b>pa_inicial_prob</b> es un diccionario que contiene probabilidades de palabras las iniciales. La palabra seleccionada se agrega a <b></b>oracion</b>.
    <div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 230px">
        <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: left;">
            pal_0
            <span style='color:fuchsia;'>=</span> 
            palabra_ejemplo(pa_inicial_prob,
            <span style='color:green;'>False</span>
            )
            <br>
            oracion.
            <span style='color:blue;'>append</span>
            (pal_0)
        </div>
    </div>   
</li>
<hr>
<li>Se elige la segunda palabra, dependiendo de la primera <b>(pal_0)</b>. <b>primer_orden_prob</b> es el diccionario que mapea palabras iniciales a posibles segundas palabras con sus probabilidades. <b>palabra_ejemplo(primer_orden_prob[pal_0], False)</b> selecciona la segunda palabra basada en <b>pal_0</b>. Se agrega <b>pal_1</b> a <b>oracion</b>.
    <div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 180px">
        <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: left;">
            pal_1
            <span style='color:fuchsia;'>=</span> 
            palabra_ejemplo(primer_orden_prob
            <span style='color:brown;'>[</span>
            pal_0
            <span style='color:brown;'>]</span>,
            <span style='color:green;'>False</span>
            )
            <br>
            oracion.
            <span style='color:blue;'>append</span>
            (pal_1)
        </div>
    </div> 
</li>
<hr>
<li>Se entra en un bucle while True para generar palabras de manera recurrente. <b>segundo_orden_prob</b> es un diccionario que usa tuplas <b>(pal_0, pal_1)</b> como clave y devuelve las probabilidades de la siguiente palabra. Se selecciona la siguiente palabra <b>(pal_2)</b> en función de las dos palabras anteriores.
    <div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 110px">
        <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: left;">
            <span style='color:blue;'>while True</span>:<br>
            <span style='color:black; margin-left:20px;'>pal_2</span> 
            <span style='color:fuchsia;'>=</span> 
            palabra_ejemplo(segundo_orden_prob
            <span style='color:brown;'>[</span>
            (pal_0, pal_1)
            <span style='color:brown;'>]</span>,
            <span style='color:green;'>False</span>
            )
        </div>
    </div>    
</li>
<hr>
<li>Si <b>pal_2</b> es <b>'END'</b>, se termina la generación de palabras y se sale del bucle.
    <div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 350px">
        <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: center;">
            <span style='color:green;'>if</span>
            pal_2
            <span style='color:fuchsia;'> == </span> 
            <span style='color:red;'>'END'</span>:<br>
            <span style='color:green; margin-left:20px;'>break</span>            
        </div>
    </div> 
</li>
<hr>
<li>Si pal_2 no es <b>'END'</b>, se agrega a la lista oracion. Luego, se actualizan las variables: <b>pal_0 toma el valor de pal_1</b> y <b>pal_1 toma el valor de pal_2</b>. Esto asegura que en la siguiente iteración se utilicen las dos palabras más recientes para predecir la siguiente.
    <div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 320px">
        <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: left;">            
            oracion.
            <span style='color:blue;'>append</span>
            <span style='color:black;'>(pal_2)</span><br>
            <span style='color:black;'>pal_0</span>
            <span style='color:fuchsia;'>=</span>
            <span style='color:black;'>pal_1</span><br>
            <span style='color:black;'>pal_1</span>
            <span style='color:fuchsia;'>=</span>
            <span style='color:black;'>pal_2</span>
        </div>
    </div>
</li>
<hr>
<li>La lista oracion se convierte en una cadena de texto separada por espacios con <b>'&nbsp;&nbsp;'.join(oracion)</b>. Se imprime la oración generada.
    <div style="display: flex; justify-content: center; align-items: center; font-family: Arial, sans-serif; margin-top: 20px; margin-left: 320px">
        <div style="padding: 10px; border: 2px solid #f1c40f; background: #fef9e7; border-radius: 5px; text-align: left;">            
            texto
            <span style='color:fuchsia;'>=</span>
            <span style='color:red;'>'&nbsp;&nbsp;'</span>.
            <span style='color:blue;'>join</span>
            <span style='color:black;'>(oracion)</span><br>
            <span style='color:green; margin-left:20px;'>print</span>
            <span style='color:black;'>(texto)</span><br>            
        </div>
    </div>
</li>
<hr>
</ul>

In [180]:
# Probamos la creación de un poema, de acuerdo con la manera de escritura de Mario Benedetti
generador(5)

mi nombre con su hermano el insociable
en esas noches en que fui un viejo cargado de recelos
mientras los g randes temas
la muerte
pero no viceversa por algo en el aire que absorbieron noche a noche


## Conclusiones

<div style="text-align: center;">
    <p>Aquí construimos un generador de texto basado en modelos de Markov de segundo orden. usando probabilidades de aparición de palabras para construir oraciones de manera secuencial hasta encontrar una palabra de terminación ('END').</p>
    <hr/>
    <p style="text-align: right;">Mg. Luis Felipe Bustamante Narváez</p>
</div>